<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/Sequential/03_workshop_binary_clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_csv('https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_clasification_binary_multiclass/sonar_csv.csv')
data_values = data.values

In [54]:
def base_model():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'glorot_uniform',activation='relu'))
  model.add(Dense(1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

# Split

In [24]:
X = data_values[:,0:60].astype('float')
y = data_values[:,60]

# Encoder

In [18]:
encoder = LabelEncoder()
encoder_y = encoder.fit_transform(y)
encoder_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
estimator = KerasClassifier(build_fn=create_model, epochs=100, batch_size = 5, verbose = 0)

kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoder_y, cv = kfold)

In [26]:
print(" Line base: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 Line base: 82.74% (7.96%)


# Optimizar el rendimiento con procesamiento de datos

In [53]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [55]:
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('mlp', KerasClassifier(build_fn=base_model, epochs = 100, batch_size = 5, verbose = 0)))

In [56]:
pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv = kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 85.10% (7.54%)


# Topologia Pequeña

In [58]:
def create_smaller():
  model = Sequential()
  model.add(Dense(30, input_dim=60,kernel_initializer = 'glorot_uniform',activation='relu'))
  model.add(Dense(1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))

  model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [60]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size = 5, verbose = 0)))

pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv = kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 86.57% (6.30%)


# Topologia grande

In [62]:
def create_large():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'glorot_uniform', activation = 'relu'))
  model.add(Dense(30, kernel_initializer = 'glorot_uniform', activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [63]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_large, epochs=100, batch_size = 5, verbose = 0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv = kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 89.40% (5.18%)


- La topología grande es la que mejor resultado tubo con respecto al resto de estructuras probadas, ya que la media de su accuracy es la más alta y su desviación es la más cercana. 

- Mucha dispersión frente a la media en los resultados no es recomendado.